In [1]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as ureq
from selenium import webdriver
import time
import re

In [2]:
url = "https://www.ulaval.ca/etudes/programmes/baccalaureat-en-genie-mecanique#section-structure"

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--incognito')
#chrome_options.add_argument('--headless')

driver = webdriver.Chrome("C:\\Users\\jerry\\Downloads\\chromedriver", options=chrome_options)

In [4]:
driver.get(url)

# Robotique

## 1. Collect course link texts for webdriver to click on

In [5]:
page_soup = soup(driver.page_source, 'lxml')

In [6]:
containers = page_soup.find("section", {"id": "section-structure"}).findAll("div", {"class": "fe-bloc-section"})[1].findAll("div", {"class": "collapsible-sections icon-first light-1 initialized"})[1].findAll("a")
len(containers)

11

In [7]:
link_texts = [container.text.strip() for container in containers]
link_texts

['GMC-3351\nÉléments de robotique',
 'GEL-4250\nCommande des systèmes avancée',
 'GIF-1003\nProgrammation avancée en C++',
 "GIF-4101\nIntroduction à l'apprentissage automatique",
 'GLO-4001\nIntroduction à la robotique mobile',
 'GMC-3300\nMécatronique',
 'GMC-3551\nProjet spécialisé',
 'GMC-4048\nAnalyse et synthèse cinématique des mécanismes',
 'GMC-4100\nCommandes hydrauliques et pneumatiques',
 'GMC-4200\nFabrication assistée par ordinateur',
 'ANL-2020']

## 2. automation script to scrape all courses

In [11]:
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.keys import Keys
tab = driver.find_element_by_id("robotique-cours-obligatoires")
time.sleep(3)
try:
    tab.click()
except ElementClickInterceptedException:
    driver.find_element_by_tag_name('body').send_keys(Keys.DOWN)
    tab.click()

In [14]:
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.keys import Keys
tab = driver.find_element_by_id("robotique-regle-1")
time.sleep(3)
try:
    tab.click()
except ElementClickInterceptedException:
    driver.find_element_by_tag_name('body').send_keys(Keys.DOWN)
    tab.click()

In [15]:
tab = driver.find_element_by_id("robotique-regle-2")
time.sleep(3)
try:
    tab.click()
except ElementClickInterceptedException:
    driver.find_element_by_tag_name('body').send_keys(Keys.DOWN)
    tab.click()

In [16]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException

course_codes = []
course_names = []
course_descs = []
counter = 0
    
for link_text in link_texts:
    #go to course page
    try:
        link = driver.find_element_by_link_text(link_text)
    except NoSuchElementException:
        print("no link for {}".format(link_text))
        continue

    time.sleep(2)
    try:
        link.click()
    except ElementClickInterceptedException:
        link.send_keys(Keys.DOWN)
        link.click()
        
    time.sleep(3)
    
    #scrape course info
    page_soup = soup(driver.page_source, 'lxml')
    course_title_container = page_soup.find("h1", {"class": "fe--titre"})
    course_code = course_title_container.find("span", {"class": "fe--titre-type"}).text.strip()
    course_name = course_title_container.find("span", {"class": "fe--titre-nom"}).text.strip()
    course_desc = page_soup.find("p", {"class": "fe--intro"}).text.strip()
    
    course_codes.append(course_code)
    course_names.append(course_name)
    course_descs.append(course_desc)
    
    print("Scraped ", course_codes[-1], course_names[-1], "\n", course_descs[-1])
    counter += 1    
    print("{} courses scraped".format(counter))
    print()
    
    #go to course list page
    driver.back()
    time.sleep(3)
        
print("Finished scraping {} courses".format(counter))

Scraped  GMC-3351 Éléments de robotique 
 Introduction. Rappels de cinématique et de dynamique, mouvement des corps rigides dans l'espace. Modèles géométrique, cinématique et dynamique des manipulateurs. Planification de trajectoires et de tâches à réaliser. Principes de la commande d'un robot.
1 courses scraped

Scraped  GEL-4250 Commande des systèmes avancée 
 Représentation d'état : représentation continue et discrète, propriétés, observabilité et gouvernabilité. Observateurs : Luenberger, filtre de Kalman. Commande modale : continue et discrète, ajout d'une action intégrale, ajout d'un observateur. Commande linéaire quadratique (LQ) : continue et discrète, ajout d'une action intégrale, ajout d'un filtre de Kalman (commande LQ Gaussienne). Commande prédictive : principe de l'horizon fuyant, critère de commande, équations de prédiction, paramètres de réglage, contraintes. Commande robuste : performances fréquentielles, incertitudes, stabilité et performance robustes, design des régul

## 3. Inspect and write to CSV

In [17]:
course_codes

['GMC-3351',
 'GEL-4250',
 'GIF-1003',
 'GIF-4101',
 'GLO-4001',
 'GMC-3300',
 'GMC-3551',
 'GMC-4048',
 'GMC-4100',
 'GMC-4200',
 'ANL-2020']

In [18]:
course_names

['Éléments de robotique',
 'Commande des systèmes avancée',
 'Programmation avancée en C++',
 "Introduction à l'apprentissage automatique",
 'Introduction à la robotique mobile',
 'Mécatronique',
 'Projet spécialisé',
 'Analyse et synthèse cinématique des mécanismes',
 'Commandes hydrauliques et pneumatiques',
 'Fabrication assistée par ordinateur',
 'Intermediate English II']

In [19]:
course_descs

["Introduction. Rappels de cinématique et de dynamique, mouvement des corps rigides dans l'espace. Modèles géométrique, cinématique et dynamique des manipulateurs. Planification de trajectoires et de tâches à réaliser. Principes de la commande d'un robot.",
 "Représentation d'état\xa0: représentation continue et discrète, propriétés, observabilité et gouvernabilité. Observateurs\xa0: Luenberger, filtre de Kalman. Commande modale\xa0: continue et discrète, ajout d'une action intégrale, ajout d'un observateur. Commande linéaire quadratique (LQ)\xa0: continue et discrète, ajout d'une action intégrale, ajout d'un filtre de Kalman (commande LQ Gaussienne). Commande prédictive\xa0: principe de l'horizon fuyant, critère de commande, équations de prédiction, paramètres de réglage, contraintes. Commande robuste\xa0: performances fréquentielles, incertitudes, stabilité et performance robustes, design des régulateurs.",
 'Programmation orientée objet en C++. Éléments de syntaxe et sémantique. Imp

In [20]:
len(list(dict.fromkeys(course_codes)))

11

In [21]:
import pandas as pd

df = pd.DataFrame({
    "Course Number": course_codes,
    "Course Name": course_names,
    "Course Description": course_descs
})

df

,Course Number,Course Name,Course Description
0,GMC-3351,Éléments de robotique,Introduction. Rappels de cinématique et de dyn...
1,GEL-4250,Commande des systèmes avancée,Représentation d'état : représentation continu...
2,GIF-1003,Programmation avancée en C++,Programmation orientée objet en C++. Éléments ...
3,GIF-4101,Introduction à l'apprentissage automatique,Ce cours porte sur les méthodes permettant l'i...
4,GLO-4001,Introduction à la robotique mobile,Ce cours offre un aperçu des principes générau...
5,GMC-3300,Mécatronique,Apprentissage des concepts de la mécatronique ...
6,GMC-3551,Projet spécialisé,Projet permettant de faire la synthèse ou d'ap...
7,GMC-4048,Analyse et synthèse cinématique des mécanismes,"Classification des mécanismes, mobilité et con..."
8,GMC-4100,Commandes hydrauliques et pneumatiques,Identification des composantes d'un système hy...
9,GMC-4200,Fabrication assistée par ordinateur,Automatisation de la fabrication. Machines-out...


In [22]:
df.to_csv('Laval_GenieMecanique_Robotique_Option_Courses.csv', index = False)

In [23]:
driver.quit()

# Génie du bâtiment durable

## 1. Collect course link texts for webdriver to click on

In [24]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--incognito')
#chrome_options.add_argument('--headless')

driver = webdriver.Chrome("C:\\Users\\jerry\\Downloads\\chromedriver", options=chrome_options)

In [25]:
driver.get(url)

In [26]:
page_soup = soup(driver.page_source, 'lxml')

In [27]:
containers = page_soup.find("section", {"id": "section-structure"}).findAll("div", {"class": "fe-bloc-section"})[1].findAll("div", {"class": "collapsible-sections icon-first light-1 initialized"})[2].findAll("a")
len(containers)

16

In [28]:
link_texts = [container.text.strip() for container in containers]
link_texts

['GMC-3012\nIntroduction à la mécanique du bâtiment et aux énergies renouvelables',
 'GMC-3015\nConception intégrée des systèmes CVAC-R',
 'ARC-2001\nArchitecture et environnement',
 'ARC-3103\nAcoustique architecturale',
 'GBO-2040\nCharpentes en bois I',
 'GBO-4070\nEnveloppe du bâtiment',
 'GCI-2004\nStructures de béton',
 'GCI-2007\nStructures métalliques',
 'GCI-3001\nImpacts environnementaux',
 'GCI-3002\nGestion des eaux',
 'GEL-2003\nÉlectrotechnique',
 'GEL-4100\nCommande industrielle',
 "GEL-4799\nDangers de l'électricité",
 'GMC-3551\nProjet spécialisé',
 'GMC-4100\nCommandes hydrauliques et pneumatiques',
 'ANL-2020']

## 2. automation script to scrape all courses

In [29]:
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.keys import Keys
tab = driver.find_element_by_id("genie-du-batiment-durable-cours-obligatoires")
time.sleep(3)
try:
    tab.click()
except ElementClickInterceptedException:
    driver.find_element_by_tag_name('body').send_keys(Keys.DOWN)
    tab.click()

In [30]:
tab = driver.find_element_by_id("genie-du-batiment-durable-regle-1")
time.sleep(3)
try:
    tab.click()
except ElementClickInterceptedException:
    driver.find_element_by_tag_name('body').send_keys(Keys.DOWN)
    tab.click()

In [31]:
tab = driver.find_element_by_id("genie-du-batiment-durable-regle-2")
time.sleep(3)
try:
    tab.click()
except ElementClickInterceptedException:
    driver.find_element_by_tag_name('body').send_keys(Keys.DOWN)
    tab.click()

In [32]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException

course_codes = []
course_names = []
course_descs = []
counter = 0
    
for link_text in link_texts:
    #go to course page
    try:
        link = driver.find_element_by_link_text(link_text)
    except NoSuchElementException:
        print("no link for {}".format(link_text))
        continue

    time.sleep(2)
    try:
        link.click()
    except ElementClickInterceptedException:
        link.send_keys(Keys.DOWN)
        link.click()
        
    time.sleep(3)
    
    #scrape course info
    page_soup = soup(driver.page_source, 'lxml')
    course_title_container = page_soup.find("h1", {"class": "fe--titre"})
    course_code = course_title_container.find("span", {"class": "fe--titre-type"}).text.strip()
    course_name = course_title_container.find("span", {"class": "fe--titre-nom"}).text.strip()
    course_desc = page_soup.find("p", {"class": "fe--intro"}).text.strip()
    
    course_codes.append(course_code)
    course_names.append(course_name)
    course_descs.append(course_desc)
    
    print("Scraped ", course_codes[-1], course_names[-1], "\n", course_descs[-1])
    counter += 1    
    print("{} courses scraped".format(counter))
    print()
    
    #go to course list page
    driver.back()
    time.sleep(3)
        
print("Finished scraping {} courses".format(counter))

Scraped  GMC-3012 Introduction à la mécanique du bâtiment et aux énergies renouvelables 
 Ce cours constitue une introduction à la mécanique du bâtiment en abordant les thèmes suivants : conditionnement de l’air, air humide, confort thermique, qualité de l’air intérieur, thermique du bâtiment, charge de chauffage, rayonnement solaire, charge de refroidissement, simulation énergétique, réseaux de distribution de fluide, éclairage. De plus, une partie du cours est consacrée aux énergies renouvelables, en particulier à la géothermie et à l’énergie solaire. Finalement, le cours propose à l'étudiant de réfléchir sur l’importance de l’énergie dans nos vies, dans un contexte de développement durable.
1 courses scraped

Scraped  GMC-3015 Conception intégrée des systèmes CVAC-R 
 Ce cours permet de mettre en application les principes du chauffage, de la ventilation, du conditionnement de l’air et de la réfrigération dans le cadre d’études de cas, d’analyse et de projets de conception. Il aborde

Finished scraping 16 courses


## 3. Inspect and write to CSV

In [33]:
course_codes

['GMC-3012',
 'GMC-3015',
 'ARC-2001',
 'ARC-3103',
 'GBO-2040',
 'GBO-4070',
 'GCI-2004',
 'GCI-2007',
 'GCI-3001',
 'GCI-3002',
 'GEL-2003',
 'GEL-4100',
 'GEL-4799',
 'GMC-3551',
 'GMC-4100',
 'ANL-2020']

In [34]:
course_names

['Introduction à la mécanique du bâtiment et aux énergies renouvelables',
 'Conception intégrée des systèmes CVAC-R',
 'Architecture et environnement',
 'Acoustique architecturale',
 'Charpentes en bois I',
 'Enveloppe du bâtiment',
 'Structures de béton',
 'Structures métalliques',
 'Impacts environnementaux',
 'Gestion des eaux',
 'Électrotechnique',
 'Commande industrielle',
 "Dangers de l'électricité",
 'Projet spécialisé',
 'Commandes hydrauliques et pneumatiques',
 'Intermediate English II']

In [35]:
course_descs

["Ce cours constitue une introduction à la mécanique du bâtiment en abordant les thèmes suivants\xa0: conditionnement de l’air, air humide, confort thermique, qualité de l’air intérieur, thermique du bâtiment, charge de chauffage, rayonnement solaire, charge de refroidissement, simulation énergétique, réseaux de distribution de fluide, éclairage. De plus, une partie du cours est consacrée aux énergies renouvelables, en particulier à la géothermie et à l’énergie solaire. Finalement, le cours propose à l'étudiant de réfléchir sur l’importance de l’énergie dans nos vies, dans un contexte de développement durable.",
 'Ce cours permet de mettre en application les principes du chauffage, de la ventilation, du conditionnement de l’air et de la réfrigération dans le cadre d’études de cas, d’analyse et de projets de conception. Il aborde des concepts et des stratégies liés à l’efficacité énergétique et au développement durable. L’étudiant apprend également à concevoir ou à choisir les principau

In [36]:
len(list(dict.fromkeys(course_codes)))

16

In [37]:
import pandas as pd

df = pd.DataFrame({
    "Course Number": course_codes,
    "Course Name": course_names,
    "Course Description": course_descs
})

df

,Course Number,Course Name,Course Description
0,GMC-3012,Introduction à la mécanique du bâtiment et aux...,Ce cours constitue une introduction à la mécan...
1,GMC-3015,Conception intégrée des systèmes CVAC-R,Ce cours permet de mettre en application les p...
2,ARC-2001,Architecture et environnement,Ce cours présente les notions fondamentales de...
3,ARC-3103,Acoustique architecturale,Ce cours vise la compréhension des principes a...
4,GBO-2040,Charpentes en bois I,"Notions préliminaires : formes structurales, p..."
5,GBO-4070,Enveloppe du bâtiment,Ce cours comprend les fondements de l'envelopp...
6,GCI-2004,Structures de béton,Théorie linéaire. Théorie à la rupture : règle...
7,GCI-2007,Structures métalliques,Introduction et bases de calcul des charpentes...
8,GCI-3001,Impacts environnementaux,Ce cours permet de s'initier aux études d'impa...
9,GCI-3002,Gestion des eaux,Gestion des eaux par bassin versant et gestion...


In [38]:
df.to_csv('Laval_GenieMecanique_BatimentDurable_Option_Courses.csv', index = False)

# Profil entrepreneurial

## 1. Collect course link texts for webdriver to click on

In [39]:
page_soup = soup(driver.page_source, 'lxml')

In [42]:
containers = page_soup.find("section", {"id": "section-structure"}).findAll("div", {"class": "fe-bloc-section"})[2].findAll("div", {"class": "collapsible-sections icon-first light-1 initialized"})[0].findAll("a")
len(containers)

4

In [43]:
link_texts = [container.text.strip() for container in containers]
link_texts

["ENT-1000\nSavoir entreprendre : la passion de créer et d'agir",
 'ENT-2023\nEntrepreneuriat technologique',
 'ENT-3000\nPortfolio entrepreneurial I',
 'ENT-3010\nPortfolio entrepreneurial II']

## 2. automation script to scrape all courses

In [44]:
tab = driver.find_element_by_id("profil-entrepreneurial-cours-obligatoires")
time.sleep(3)
try:
    tab.click()
except ElementClickInterceptedException:
    driver.find_element_by_tag_name('body').send_keys(Keys.DOWN)
    tab.click()

In [45]:
course_codes = []
course_names = []
course_descs = []
counter = 0
    
for link_text in link_texts:
    #go to course page
    try:
        link = driver.find_element_by_link_text(link_text)
    except NoSuchElementException:
        print("no link for {}".format(link_text))
        continue

    time.sleep(2)
    try:
        link.click()
    except ElementClickInterceptedException:
        link.send_keys(Keys.DOWN)
        link.click()
        
    time.sleep(3)
    
    #scrape course info
    page_soup = soup(driver.page_source, 'lxml')
    course_title_container = page_soup.find("h1", {"class": "fe--titre"})
    course_code = course_title_container.find("span", {"class": "fe--titre-type"}).text.strip()
    course_name = course_title_container.find("span", {"class": "fe--titre-nom"}).text.strip()
    course_desc = page_soup.find("p", {"class": "fe--intro"}).text.strip()
    
    course_codes.append(course_code)
    course_names.append(course_name)
    course_descs.append(course_desc)
    
    print("Scraped ", course_codes[-1], course_names[-1], "\n", course_descs[-1])
    counter += 1    
    print("{} courses scraped".format(counter))
    print()
    
    #go to course list page
    driver.back()
    time.sleep(3)
        
print("Finished scraping {} courses".format(counter))

Scraped  ENT-1000 Savoir entreprendre : la passion de créer et d'agir 
 Ce cours d'introduction à l'entrepreneuriat s'adresse à tout étudiant inscrit au premier cycle. Il vise à développer le sens de l'initiative et l'esprit d'entreprise chez l'étudiant, afin de lui faire découvrir et exploiter son plein potentiel entrepreneurial. Les connaissances transmises à l'étudiant concernent à la fois l'entrepreneur et le processus créatif par lequel il mène son projet à terme, qu'il s'agisse d'un projet social, commercial, coopératif, artistique ou autre. Des exemples concrets sont présentés pour illustrer la passion qui anime les entrepreneurs, leur besoin de créer et d'innover et leur orientation vers l'action. Offert en classe ou à distance, ce cours joue un rôle clé dans le développement des compétences entrepreneuriales. Le cours inclut la réalisation d'une esquisse entrepreneuriale afin de sensibiliser l'étudiant aux réalités d'un projet entrepreneurial et développer une vision plus just

## 3. Inspect and write to CSV

In [46]:
course_codes

['ENT-1000', 'ENT-2023', 'ENT-3000', 'ENT-3010']

In [47]:
course_names

["Savoir entreprendre : la passion de créer et d'agir",
 'Entrepreneuriat technologique',
 'Portfolio entrepreneurial I',
 'Portfolio entrepreneurial II']

In [48]:
course_descs

["Ce cours d'introduction à l'entrepreneuriat s'adresse à tout étudiant inscrit au premier cycle. Il vise à développer le sens de l'initiative et l'esprit d'entreprise chez l'étudiant, afin de lui faire découvrir et exploiter son plein potentiel entrepreneurial. Les connaissances transmises à l'étudiant concernent à la fois l'entrepreneur et le processus créatif par lequel il mène son projet à terme, qu'il s'agisse d'un projet social, commercial, coopératif, artistique ou autre. Des exemples concrets sont présentés pour illustrer la passion qui anime les entrepreneurs, leur besoin de créer et d'innover et leur orientation vers l'action. Offert en classe ou à distance, ce cours joue un rôle clé dans le développement des compétences entrepreneuriales. Le cours inclut la réalisation d'une esquisse entrepreneuriale afin de sensibiliser l'étudiant aux réalités d'un projet entrepreneurial et développer une vision plus juste des défis à considérer. L'esquisse n'étant pas un plan d'affaires co

In [49]:
import pandas as pd

df = pd.DataFrame({
    "Course Number": course_codes,
    "Course Name": course_names,
    "Course Description": course_descs
})

df

,Course Number,Course Name,Course Description
0,ENT-1000,Savoir entreprendre : la passion de créer et d...,Ce cours d'introduction à l'entrepreneuriat s'...
1,ENT-2023,Entrepreneuriat technologique,Ce cours vise à acquérir une compétence distin...
2,ENT-3000,Portfolio entrepreneurial I,Préalables : être inscrit au profil entreprene...
3,ENT-3010,Portfolio entrepreneurial II,Le portfolio entrepreneurial permet à l'étudia...


In [50]:
df.to_csv('Laval_GenieMecanique_entrepreneurial_profile_Courses.csv', index = False)

# Profil international

## 1. Collect course link texts for webdriver to click on

In [51]:
page_soup = soup(driver.page_source, 'lxml')

In [52]:
containers = page_soup.find("section", {"id": "section-structure"}).findAll("div", {"class": "fe-bloc-section"})[2].findAll("div", {"class": "collapsible-sections icon-first light-1 initialized"})[1].findAll("a")
len(containers)

1

In [53]:
link_texts = [container.text.strip() for container in containers]
link_texts

['EHE-1GMC\nÉtudes - Profil international - Baccalauréat en génie mécanique']

## 2. scrape course

In [55]:
tab = driver.find_element_by_id("profil-international-cours-obligatoires")
time.sleep(3)
try:
    tab.click()
except ElementClickInterceptedException:
    driver.find_element_by_tag_name('body').send_keys(Keys.DOWN)
    tab.click()

In [58]:
course_codes = []
course_names = []
course_descs = []
counter = 0
    
for link_text in link_texts:
    #go to course page
    try:
        link = driver.find_element_by_link_text(link_text)
    except NoSuchElementException:
        print("no link for {}".format(link_text))
        continue

    time.sleep(2)
    try:
        link.click()
    except ElementClickInterceptedException:
        link.send_keys(Keys.DOWN)
        link.click()
        
    time.sleep(3)
    
    #scrape course info
    page_soup = soup(driver.page_source, 'lxml')
    course_code = link_text.split("\n")[0]
    course_name = link_text.split("\n")[1]
    course_desc = page_soup.find("div", {"class": "fe--intro"}).text.strip()
    
    course_codes.append(course_code)
    course_names.append(course_name)
    course_descs.append(course_desc)
    
    print("Scraped ", course_codes[-1], course_names[-1], "\n", course_descs[-1])
    counter += 1    
    print("{} courses scraped".format(counter))
    print()
    
    #go to course list page
    driver.back()
    time.sleep(3)
        
print("Finished scraping {} courses".format(counter))

Scraped  EHE-1GMC Études - Profil international - Baccalauréat en génie mécanique 
 Ce baccalauréat pourrait bien être la clé pour donner libre cours à votre créativité. Le génie mécanique est partout, dans tous les aspects de l'activité humaine. Pour n'en mentionner que quelques exemples, on retrouve des ingénieurs mécaniques dans tous les secteurs de l'industrie, dans les transports, les services publics, les loisirs, l'éducation, les sports, les arts, la médecine et les sciences de la santé, les technologies environnementales, dans la conception et la réalisation de tous les objets que nous utilisons dans la vie de tous les jours.
1 courses scraped

Finished scraping 1 courses


## 3. Inspect and write to CSV

In [59]:
course_codes

['EHE-1GMC']

In [60]:
course_names

['Études - Profil international - Baccalauréat en génie mécanique']

In [61]:
course_descs

["Ce baccalauréat pourrait bien être la clé pour donner libre cours à votre créativité. Le génie mécanique est partout, dans tous les aspects de l'activité humaine. Pour n'en mentionner que quelques exemples, on retrouve des ingénieurs mécaniques dans tous les secteurs de l'industrie, dans les transports, les services publics, les loisirs, l'éducation, les sports, les arts, la médecine et les sciences de la santé, les technologies environnementales, dans la conception et la réalisation de tous les objets que nous utilisons dans la vie de tous les jours."]

In [62]:
import pandas as pd

df = pd.DataFrame({
    "Course Number": course_codes,
    "Course Name": course_names,
    "Course Description": course_descs
})

df

,Course Number,Course Name,Course Description
0,EHE-1GMC,Études - Profil international - Baccalauréat e...,Ce baccalauréat pourrait bien être la clé pour...


In [63]:
df.to_csv('Laval_GenieMecanique_international_profile_Courses.csv', index = False)

In [64]:
driver.quit()